In [ ]:
!pip install llama-index.core pinecone-client openai
import os

os.environ["OPENAI_API_KEY"] = "# Replace with your actual OpenAI API key"
os.environ["PINECONE_API_KEY"] = "# Replace with you actual Pinecone API key"
os.environ["PINECONE_ENVIRONMENT"] = "us-east-1"
import os
from pinecone import Pinecone, ServerlessSpec

# Create an instance of the Pinecone class
pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)

# Create the index if it doesn't exist
index_name = "llama-index-nvidia"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # Adjust the dimension as needed
        metric='euclidean',
        spec=ServerlessSpec(
            cloud='gcp',  # Change to the cloud service you're using (e.g., 'aws', 'azure')
            region='us-west1'  # Ensure this matches your chosen environment
        )
    )

# Retrieve the index object
index = pc.Index(index_name)

!pip install llama_index.readers.web
from llama_index.readers.web import SimpleWebPageReader

urls = ["https://docs.api.nvidia.com/nim/reference/ipd-rfdiffusion"]  # Replace with your target URLs
documents = SimpleWebPageReader(html_to_text=True).load_data(urls)

!pip install llama_index.vector_stores.pinecone
!pip install llama_index.llms.openai
from llama_index.core import Settings
import os
from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.pinecone import PineconeVectorStore

# Initialize OpenAI LLM
llm = OpenAI(temperature=0, model="text-embedding-ada-002")

# Initialize Pinecone Vector Store
vector_store = PineconeVectorStore(index_name=index_name, environment=os.environ["PINECONE_ENVIRONMENT"])

# Set up storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Configure global settings
Settings.llm = llm

# Create and populate the index
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
